In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import math
import random

import numpy as np
import tensorflow as tf
import cv2

import scipy.io as sio
slim = tf.contrib.slim

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
sys.path.insert(0,'../')
sys.path.insert(0,'../processing/')
from datasets import sythtextprovider
from nets import txtbox_300

In [3]:
from nets import txtbox_300, textbox_common, np_methods
from processing import txt_preprocessing
from processing import visualization

In [4]:
isess = tf.InteractiveSession()

In [125]:
# Input placeholder.
net_shape = (300, 300)
data_format = 'NHWC'
img_input = tf.placeholder(tf.uint8, shape=(None, None, 3))
# Evaluation pre-processing: resize to SSD net shape.
image_pre, labels_pre, bboxes_pre, bbox_img = txt_preprocessing.preprocess_for_eval(
    img_input, None, None, net_shape, data_format, resize=txt_preprocessing.Resize.WARP_RESIZE)
image_4d = tf.expand_dims(image_pre, 0)

# Define the SSD model.
reuse = True if 'net' in locals() else None
net = txtbox_300.TextboxNet()
with slim.arg_scope(net.arg_scope(data_format=data_format)):
    localisations, predictions,  _ = net.net(image_4d, is_training=False, reuse=True)
    for i in range(len(rpredictions)):
        predictions[i] = tf.nn.softmax(predictions[i])
        


In [126]:
# Restore SSD model.
ckpt_filename = '../checkpoints/model.ckpt-1337.meta'
# ckpt_filename = '../checkpoints/VGG_VOC0712_SSD_300x300_ft_iter_120000.ckpt'
isess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
#saver = tf.train.import_meta_graph(ckpt_filename)
saver.restore(isess, '../checkpoints/model.ckpt-13657')

# SSD default anchor boxes.
anchors = net.anchors(net_shape)

In [8]:
#rpredictions.shape
loss = isess.run(tf.nn.softmax(tf.constant([[0.52,0.47],[-0.52,-0.47]])))
logit_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = tf.constant([[0.52,0.47],[-0.52,-0.47]]),
                                                         labels =tf.constant([0,1]) )
print isess.run([logit_loss,loss])
print isess.run(tf.reduce_mean(logit_loss))

TypeError: Fetch argument array([[ 0.51249737,  0.48750257],
       [ 0.48750257,  0.51249737]], dtype=float32) has invalid type <type 'numpy.ndarray'>, must be a string or Tensor. (Can not convert a ndarray into a Tensor or Operation.)

In [151]:
rimg, rpredictions, rlocalisations, rbbox_img = isess.run([image_4d, predictions, localisations, bbox_img],
                                                              feed_dict={img_input: img})
rclasses, rscores, rbboxes = np_methods.ssd_bboxes_select(
            rpredictions, rlocalisations, anchors,
            select_threshold=0.5, img_shape=(300,300), num_classes=2, decode=True)




file_path: ../data/sythtext/1.tfrecord


In [178]:
for i in range(6):
    print np.where(rpredictions[i][:,:,:,:,:,1:] >0.7)
    
bboxes = np_methods.ssd_bboxes_decode(rlocalisations[0], anchors[0])
rlocalisations

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
(array([0, 0, 0]), array([12, 13, 13]), array([10,  9, 10]), array([1, 1, 1]), array([2, 2, 2]), array([0, 0, 0]))
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


[array([[[[[[  3.25859264e-02,   7.24070705e-03,   1.65571749e-01,
               3.61762084e-02],
            [ -2.11369321e-02,   7.47503489e-02,   1.75052390e-01,
               1.85265362e-01],
            [  5.87446876e-02,   9.15655494e-03,   1.74111545e-01,
               1.49492681e-01],
            [  4.73613590e-02,   3.96350138e-02,   5.76067120e-02,
               1.07092485e-01],
            [  6.63525751e-03,  -4.92215604e-02,   8.25837106e-02,
              -2.54073478e-02],
            [  7.39921480e-02,   7.33257532e-02,   6.27900362e-02,
               1.05389073e-01]],
 
           [[  6.93395659e-02,  -9.83331278e-02,   1.14711508e-01,
               7.62878954e-02],
            [  1.83063298e-02,  -8.66914168e-02,   1.91883624e-01,
               1.74271554e-01],
            [  6.98217899e-02,   9.89017915e-03,   1.33454740e-01,
               1.85348853e-01],
            [  1.04093272e-02,   6.10322803e-02,   4.08567339e-02,
               1.12395070e-01],
       

In [184]:

# Main image processing routine.
def process_image(img, select_threshold=0.5, nms_threshold=.45, net_shape=(300, 300)):
    # Run SSD network.
    rimg, rpredictions, rlocalisations, rbbox_img,image_ = isess.run([image_4d, predictions, localisations, bbox_img,image],
                                                              feed_dict={img_input: img})
    
    

    # Get classes and bboxes from the net outputs.
    rclasses, rscores, rbboxes = np_methods.ssd_bboxes_select(
            rpredictions, rlocalisations, anchors,
            select_threshold=select_threshold, img_shape=net_shape, num_classes=2, decode=True)
    
    rbboxes = np_methods.bboxes_clip(rbboxes)
    rclasses, rscores, rbboxes = np_methods.bboxes_sort(rclasses, rscores, rbboxes, top_k=400)
    rclasses, rscores, rbboxes = np_methods.bboxes_nms(rclasses, rscores, rbboxes, nms_threshold=nms_threshold)
    # Resize bboxes to original image shape. Note: useless for Resize.WARP!
    rbboxes = np_methods.bboxes_resize(rbbox_img, rbboxes)
    return rclasses, rscores, rbboxes, image_

In [ ]:
img = mpimg.imread('../data/ICDAR2013/ICDAR-Test-Images/img_101.jpg')
rclasses, rscores, rbboxes,img =  process_image(img)

# visualization.bboxes_draw_on_img(img, rclasses, rscores, rbboxes, visualization.colors_plasma)
visualization.plt_bboxes(img, rclasses, rscores, rbboxes)